In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [ ]:
# 定义参数
year = '2012'
startDate = year + '-01-01'
endDate = year + '-12-31'

In [ ]:
# 武汉市
# roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市',city))
roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [ ]:
# 移除无效像元
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

## 指数的计算
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi=image.normalizedDifference(['B3', 'B5']).rename("NDWI")
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    cwi=image.select('B3').divide(image.select('B6')).rename("CWI")
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    ewi = image.expression('(B3 - B5 - B6)/(B3 + B5 + B6)',
        {
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
        }).rename('EWI')
    evi = image.expression('2.5*(B5 - B4)/(B5 + 6*B4 - 7.5*B2 + 1)',
        {
          'B2': image.select('B2'),
          'B4': image.select('B4'),
          'B5': image.select('B5'),    
        }).rename('EVI')
    return image.addBands(ndvi).addBands(ndwi).addBands(mndwi).addBands(cwi).addBands(awei).addBands(ewi).addBands(evi).addBands(ndvi_mndwi)

# 计算研究区域内，影像面积
def ImageArea(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': roi,
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})


In [ ]:
l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
           .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
           .filterBounds(roi) \
           .filterDate(startDate, endDate)
           
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
       .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'],['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate(startDate, endDate)          
landsat_images = l5.merge(l7).map(maskSR).map(water_index)
Map.addLayer(l5.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
print(l7.size().getInfo())
print(l5.size().getInfo())
print(landsat_images.size().getInfo())

# 筛选得出采样影像

In [ ]:
# 筛选图像
image_area = landsat_images.mosaic().select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14
})
print(areas.getInfo())
# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(ImageArea).filter(ee.Filter.gt('imagearea',ee.Number(areas.get('B2')).divide(3)))
dates = sample_image.aggregate_array('system:time_start').sort().map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

# 多水体频率数据集的类别范围确定

In [ ]:
#### 设置年份
JRC_id = 'JRC/GSW1_3/YearlyHistory/' + year
Maryland_id = 'users/311605001111/Maryland/Maryland_nationwide_' + year
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC = ee.Image(JRC_id).clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC,basemap]).sum()
# Maryland
Maryland = ee.Image(Maryland_id).clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland.gte(25),Maryland.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

# Map.addLayer(permanentWaterExtent.selfMask(),{'palette':['blue']},"permanent_water")
# Map.addLayer(seasonWaterExtent.selfMask(),{'palette':['green']},"season water")

# 采样函数

In [ ]:
### 一般阈值：AWEI=0
# 设置采样个数
number_land = 250
number_water = 500
# 基于图像的样本采集函数
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    water_points = permanent_points.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    nowater_points = land_points.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    # 季节性水体
    awei = image.updateMask(seasonWaterExtent).select('AWEI').gt(0)
    seasonWater = image.updateMask(awei).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_water,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonWater_points = seasonWater.map(lambda i : i.setMulti({'waterclass':1,'Image_id':image.get('system:id')}))
    seasonLand= image.updateMask(awei.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number_land,
        'seed': 0,
        'geometries': True,
        'tileScale': 2,
    })
    seasonLand_points = seasonLand.map(lambda i : i.setMulti({'waterclass':0,'Image_id':image.get('system:id')}))
    sample_points = ee.FeatureCollection([water_points,nowater_points,seasonWater_points,seasonLand_points]).flatten()
    return sample_points

# 基于影像，进行样本采集

In [3]:
# 总的样本点数
points_collection = sample_image.map(imageSample).flatten()
# print(points_collection.first().getInfo())
# print('total sample number:{}'.format(points_collection.size().getInfo()))

dataset_id = 'wuhan_' + year
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.table.toAsset(**{
    'collection': points_collection,
    'description': dataset_id,
    'assetId': assetID
})
task.start()

NameError: name 'sample_image' is not defined

In [ ]:
# 总的样本点数
a = sample_image
points_collection = a.map(imageSample).flatten()
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))

In [ ]:
# 水体的样本点数
samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# 非水体的样本点数
samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
print('nowater sample number:{}'.format(samplepoint_water.size().getInfo()))

In [ ]:
folder = 'wuhan'
label = 'wuhan_new' + year + '_l57_awei0'
# label = 'tianjin_' + 'new2020_l78'
# csv格式
selector = ['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder=folder, file_format='csv', selectors=selector)

# shp格式
selector = ['Image_id','B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder=folder, file_format='shp', selectors=selector)